# Computer Lab: Quantile Regression

Olivier Fercoq

# Preparation
Please read the file TPquantile_regression_beginning.pdf for instructions on how to launch this notebook

In [1]:
%pylab

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
#import findspark
#findspark.init()
#import pyspark
#sc = pyspark.SparkContext()

In [5]:
filename = './data/ethylene_methane.txt'

In [6]:
dist_data = sc.textFile(filename)
dist_data.take(2)

['Time (seconds), Methane conc (ppm), Ethylene conc (ppm), sensor readings (16 channels) ',
 '0.00    0.00    0.00    -41.98  2067.64 -37.13  2.28    8.63    -26.62  -8.46   -0.33   3437.73 2728.14 4054.03 4007.89 4478.27 5056.98 3639.09 3128.49']

In [ ]:
dist_data = dist_data.filter(lambda line: line[0:4] != 'Time')  # remove the first line
dist_data.first()

In [ ]:
s = dist_data.first()
print(s)
a = np.array(map(float, s.split()))
print(a)

In [ ]:
dist_data = dist_data.map(lambda line: np.array(map(float, line.split())) )
dist_data.first()

In [ ]:
# We take about half of the data for the training set

In [ ]:
train_data_full = dist_data.filter(lambda row: row[0] < 42082.60 / 2.)

In [ ]:
# In order to make development smoother, we first consider 1/100th of the training data. 
# This can be changed when the algorithm is operational.

In [ ]:
train_data_full.cache()  # Otherwise, all previous filter operations are re-run each time an action is taken.

freq = 100
train_data = train_data_full.filter(lambda row: int(row[0]*100) % freq == 0)
train_data.cache()  # Otherwise, all previous filter operations are re-run each time an action is taken.


In [ ]:
def standardize(train_data):
    print('before centering', train_data.first())
    row_sums = train_data.reduce(lambda a, b: a + b)
    print('nb of columns', row_sums.shape)
    n_train = train_data.count()
    print('nb of observations', n_train)

    row_means = row_sums / n_train
    print('row_means', row_means)

    train_data = train_data.map(lambda r: r - row_means)
    print('after centering', train_data.first())

    row_stds = np.sqrt(train_data.map(lambda r: r ** 2).reduce(lambda a, b: a + b) / n_train)
    print('row_stds', row_stds)
    train_data = train_data.map(lambda r: r / row_stds)
    print('after standardizing', train_data.first())

    row_sums_should_be_0 = train_data.reduce(lambda a, b: a + b)
    print('this should be 0', row_sums_should_be_0)
    row_stds_should_be_1 = np.sqrt(train_data.map(lambda r: r ** 2).reduce(lambda a, b: a + b) / n_train)
    print('this should be 1', row_stds_should_be_1)

    return train_data

train_data = standardize(train_data)
train_data_NotDistributed = np.array(train_data.collect())  # collect() should be used only on small RDDs

train_data_full = standardize(train_data_full)

# Distributed least squares